<a href="https://colab.research.google.com/github/tonyw54/GeorgeBrown/blob/main/MedQuad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset
import torch

In [ ]:
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [ ]:
# prompt: how can I train an ANN model on this dataset to create a chatbot?

# Assuming 'dataset' is your loaded dataset.

# 1. Prepare the data:
#    - Tokenize the questions and context.
#    - Create input tensors for the model.

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # Choose a suitable model

def preprocess_function(examples):
  questions = [q for q in examples["question"]]
  contexts = [c for c in examples["context"]]
  inputs = tokenizer(
      questions,
      contexts,
      max_length=384,
      truncation="only_second",
      return_offsets_mapping=True,
      padding="max_length",
  )
  return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# 2. Define the model:
#    - Choose a suitable pre-trained model for question answering.

model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")  # Choose a suitable model

# 3. Fine-tune the model:
#    - Use a suitable training loop and optimizer.

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

# Train the model
trainer.train()

# 4. Evaluate the model:
#    - Evaluate the model on the validation set.

# 5. Deploy the chatbot:
#    - Use the fine-tuned model to answer user queries.

# Example inference:
def answer_question(question, context):
  inputs = tokenizer(question, context, return_tensors="pt")
  with torch.no_grad():
    outputs = model(**inputs)
  start_logits = outputs.start_logits
  end_logits = outputs.end_logits
  start_index = torch.argmax(start_logits)
  end_index = torch.argmax(end_logits)
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))
  return answer

# Example usage:
question = "What is the cause of this disease?"
context = "This disease is caused by a virus."
answer = answer_question(question, context)
print(answer)


In [ ]:
# convert dataset into SQuAD 2.0 format
import uuid

def convert_to_squad2(dataset):
    squad2_dataset = []
    for example in dataset:
        squad2_example = {
            "title": "Medical Q&A",
            "paragraphs": [
                {
                    "context": example["Answer"],
                    "qas": [
                        {
                            "question": example["Question"],
                            "id": str(uuid.uuid4()),  # Generate a unique ID
                            "answers": [
                                {
                                    "text": example["Answer"],
                                    "answer_start": 0
                                }
                            ],
                            "is_impossible": False
                        }
                    ]
                }
            ]
        }
        squad2_dataset.append(squad2_example)
    return squad2_dataset

squad2_dataset = convert_to_squad2(dataset["train"])

In [ ]:
model_name = "deepset/roberta-base-squad2" # base model defined for Q and A
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# prompt: fine tune this  SQuAD 2.0 model  with the squad2_dataset (also SQuAD 2.0)

from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=squad2_dataset,
    #eval_dataset=squad2_dataset  # You can use a separate dataset for evaluation
)

# Train the model
trainer.train()


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,start_positions,end_positions,output_attentions,output_hidden_states,return_dict,end_positions,label,start_positions,label_ids.

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["Question"]]
    inputs = tokenizer(
        questions,
        examples["Answer"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["Answer"]

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Tokenize the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)
train_dataset

Map:   0%|          | 0/6720 [00:00<?, ? examples/s]

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

Dataset({
    features: ['qtype', 'Question', 'Answer'],
    num_rows: 6720
})

In [ ]:
# prompt: show the first 5 rows of train_dataset

print(train_dataset[:1])


{'qtype': ['inheritance'], 'Question': ['Is Kniest dysplasia inherited ?'], 'Answer': ['This condition is inherited in an autosomal dominant pattern, which means one copy of the altered gene in each cell is sufficient to cause the disorder.']}


In [ ]:
# prompt: my dataset only has qType, Question and Answer columns. Spiit dataset into train and test datasets and tokenize them. How to fine tune the model above with my dataset?



# Fine-tune the model
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


Map:   0%|          | 0/8400 [00:00<?, ? examples/s]

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

TypeError: can only join an iterable

In [ ]:
# Assuming you have a 'context' and 'question' column in your dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["Question"]]
    inputs = tokenizer(
        questions,
        examples["Answer"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    return inputs

# Now you can use the tokenizer in the map function
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Assuming you have a model defined (e.g., model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased"))
model_name = 'gbc/ml1group6' # Replace with your desired model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [ ]:
# prompt: how can I create a model from dataset?


# Fine-tune the model
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()


In [ ]:
model_name = "jiviai/biomistral-ft-10k" # This is a valid model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.mistral.configuration_mistral.MistralConfig'> for this kind of AutoModel: AutoModelForQuestionAnswering.
Model type should be one of AlbertConfig, BartConfig, BertConfig, BigBirdConfig, BigBirdPegasusConfig, BloomConfig, CamembertConfig, CanineConfig, ConvBertConfig, Data2VecTextConfig, DebertaConfig, DebertaV2Config, DistilBertConfig, ElectraConfig, ErnieConfig, ErnieMConfig, FalconConfig, FlaubertConfig, FNetConfig, FunnelConfig, GPT2Config, GPTNeoConfig, GPTNeoXConfig, GPTJConfig, IBertConfig, LayoutLMv2Config, LayoutLMv3Config, LEDConfig, LiltConfig, LlamaConfig, LongformerConfig, LukeConfig, LxmertConfig, MarkupLMConfig, MBartConfig, MegaConfig, MegatronBertConfig, MobileBertConfig, MPNetConfig, MptConfig, MraConfig, MT5Config, MvpConfig, NemotronConfig, NezhaConfig, NystromformerConfig, OPTConfig, QDQBertConfig, ReformerConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerC

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["Question"]]
    inputs = tokenizer(
        questions,
        examples["Answer"],
        padding="max_length",
        truncation=True,
        return_offsets_mapping=True,
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["Answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        if isinstance(answer, str):
            answer = [answer] # Make sure answer is a list
        start_char = answer[0].find(answer[0]) # Access the first element of the list
        end_char = start_char + len(answer[0])

        # Convert the answer's start/end positions in characters to start/end positions in tokens
        token_start_index = 0
        while token_start_index < len(offset) and offset[token_start_index][0] <= start_char:
            token_start_index += 1
        start_positions.append(token_start_index - 1)

        token_end_index = len(offset) - 1
        while token_end_index >= 0 and offset[token_end_index][1] >= end_char:
            token_end_index -= 1
        end_positions.append(token_end_index + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [ ]:
# prompt: split the dataset into train and evaluation datasets

train_dataset = dataset["train"].train_test_split(test_size=0.2)["train"]
evaluation_dataset = dataset["train"].train_test_split(test_size=0.2)["test"]


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    gradient_accumulation_steps=2,
    remove_unused_columns=False # Add this line
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=evaluation_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


The below needs to run on a GPU with maximum RAM. Even on Colab, it will take almost 15 minutes.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,0.000000,nan
2,0.000000,nan


TrainOutput(global_step=2460, training_loss=0.0, metrics={'train_runtime': 786.796, 'train_samples_per_second': 50.045, 'train_steps_per_second': 3.127, 'total_flos': 5304777167499264.0, 'train_loss': 0.0, 'epoch': 2.998171846435101})

In [ ]:
# Assuming you have already trained the model and have the 'trainer' object

# Save the model
#trainer.save_model("output/model")

# Save the tokenizer
#tokenizer.save_pretrained("output/model") # Use the tokenizer object to save it

('output/model/tokenizer_config.json',
 'output/model/special_tokens_map.json',
 'output/model/vocab.txt',
 'output/model/added_tokens.json',
 'output/model/tokenizer.json')

In [ ]:
# If the model has already been saved
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("output/model")
tokenizer = AutoTokenizer.from_pretrained("output/model")

In [ ]:
# prompt: What is "context" in answer_question?

def answer_question(question, context):
    """
    Answers a question based on the given context.

    Args:
        question: The question to answer.
        context: The text containing the answer.

    Returns:
        A string containing the answer.
    """
    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))
    return answer

# Example usage:
question = "Who is at risk for Lymphocytic Choriomeningitis (LCM)?"
context = "Individuals of all ages who come into contact with urine, feces, saliva, or blood of wild mice are potentially at risk for infection. Owners of pet mice or hamsters may be at risk for infection if these animals originate from colonies that were contaminated with LCMV, or if their animals are infected from other wild mice. Human fetuses are at risk of acquiring infection vertically from an infected mother. Laboratory workers who work with the virus or handle infected animals are also at risk. However, this risk can be minimized by utilizing animals from sources that regularly test for the virus, wearing proper protective laboratory gear, and following appropriate safety precautions."
answer = answer_question(question, context)
print(answer)


Who
